In [ ]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('kdd').getOrCreate()

In [4]:
data = spark.read.csv('Cung cap du lieu buoi 6/kdd_data/kddcup.data_10_percent.gz', inferSchema = True, header = False)

In [5]:
data.count()

494021

In [6]:
from pyspark.sql.functions import col

In [7]:
str(data.columns)

"['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22', '_c23', '_c24', '_c25', '_c26', '_c27', '_c28', '_c29', '_c30', '_c31', '_c32', '_c33', '_c34', '_c35', '_c36', '_c37', '_c38', '_c39', '_c40', '_c41']"

In [8]:
cols = ['_c0', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22', '_c23', '_c24', '_c25', '_c26', '_c27', '_c28', '_c29', '_c30', '_c31', '_c32', '_c33', '_c34', '_c35', '_c36', '_c37', '_c38', '_c39', '_c40']

for col_name in cols:
    data = data.withColumn(col_name, col(col_name).cast('float'))

In [9]:
data.show(3)

+---+---+----+---+-----+------+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-------+
|_c0|_c1| _c2|_c3|  _c4|   _c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|_c28|_c29|_c30|_c31|_c32|_c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|   _c41|
+---+---+----+---+-----+------+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-------+
|0.0|tcp|http| SF|181.0|5450.0|0.0|0.0|0.0|0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 8.0| 8.0| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0| 9.0| 9.0| 1.0| 0.0|0.11| 0.0| 0.0| 0.0| 0.0| 0.0|normal.|
|0.0|tcp|http| SF|239.0| 486.0|0.0|0.0|0.0|0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 8.0| 8.0| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0|19.0

In [10]:
data.groupBy('_c41').count().show(30)

+----------------+------+
|            _c41| count|
+----------------+------+
|    warezmaster.|    20|
|          smurf.|280790|
|            pod.|   264|
|           imap.|    12|
|           nmap.|   231|
|   guess_passwd.|    53|
|        ipsweep.|  1247|
|      portsweep.|  1040|
|          satan.|  1589|
|           land.|    21|
|     loadmodule.|     9|
|      ftp_write.|     8|
|buffer_overflow.|    30|
|        rootkit.|    10|
|    warezclient.|  1020|
|       teardrop.|   979|
|           perl.|     3|
|            phf.|     4|
|       multihop.|     7|
|        neptune.|107201|
|           back.|  2203|
|            spy.|     2|
|         normal.| 97278|
+----------------+------+



In [11]:
str(data.columns)

"['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22', '_c23', '_c24', '_c25', '_c26', '_c27', '_c28', '_c29', '_c30', '_c31', '_c32', '_c33', '_c34', '_c35', '_c36', '_c37', '_c38', '_c39', '_c40', '_c41']"

In [12]:
train_data, test_data = data.randomSplit([0.8, 0.2])

In [13]:
test_data.groupBy('_c41').count().show(30)

+----------------+-----+
|            _c41|count|
+----------------+-----+
|    warezmaster.|    3|
|          smurf.|56551|
|            pod.|   45|
|           nmap.|   47|
|           imap.|    4|
|   guess_passwd.|   11|
|        ipsweep.|  260|
|      portsweep.|  216|
|          satan.|  313|
|           land.|    7|
|     loadmodule.|    3|
|      ftp_write.|    3|
|buffer_overflow.|    7|
|        rootkit.|    2|
|    warezclient.|  222|
|       teardrop.|  174|
|           perl.|    1|
|            phf.|    2|
|       multihop.|    1|
|        neptune.|21411|
|           back.|  485|
|         normal.|19265|
+----------------+-----+



In [14]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator

In [15]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [16]:
# Import class for creating a pipeline
from pyspark.ml import Pipeline

In [17]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier

In [18]:
# Convert categorical strings to index values
indexer1 = StringIndexer(inputCol='_c1', outputCol='c1_idx')
indexer2 = StringIndexer(inputCol='_c2', outputCol='c2_idx')
indexer3 = StringIndexer(inputCol='_c3', outputCol='c3_idx')
indexer41 = StringIndexer(inputCol='_c41', outputCol='c41_idx')

# One-hot encode index values
onehot = OneHotEncoderEstimator(
    inputCols=['c1_idx', 'c2_idx', 'c3_idx'],
    outputCols=['c1_dummy', 'c2_dummy', 'c3_dummy'])

# Assemble predictors into a single column
assembler = VectorAssembler(
    inputCols=['_c0', 'c1_dummy', 'c2_dummy', 'c3_dummy', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10',
               '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22', 
               '_c23', '_c24', '_c25', '_c26', '_c27', '_c28', '_c29', '_c30', '_c31', '_c32', '_c33', '_c34', 
               '_c35', '_c36', '_c37', '_c38', '_c39', '_c40'],
    outputCol='features')

# A linear regression object
dtc = DecisionTreeClassifier(featuresCol='features', 
                             labelCol='c41_idx',
                             predictionCol='prediction')

In [19]:
# Construct a pipeline
pipeline = Pipeline(stages=[indexer1, indexer2, indexer3, 
                            indexer41, onehot, assembler, dtc])

In [20]:
# Train the pipeline on the training data
pipeline = pipeline.fit(train_data)

In [21]:
# Make predictions on the testing data
predictions = pipeline.transform(test_data)

In [22]:
# Inspect results
predictions.select('prediction', 'c41_idx').show(5)

+----------+-------+
|prediction|c41_idx|
+----------+-------+
|       2.0|    5.0|
|       2.0|   10.0|
|       2.0|    5.0|
|       2.0|   10.0|
|       2.0|    5.0|
+----------+-------+
only showing top 5 rows



In [23]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [24]:
# Select (prediction, true label) and compute test error
acc_evaluator = MulticlassClassificationEvaluator(labelCol='c41_idx',
                                                  predictionCol='prediction',
                                                  metricName='accuracy')

In [26]:
# Important: need to cast to float type, and order by prediction, else it won't work
preds_and_labels = predictions.select(['prediction', 'c41_idx'])\
                        .withColumn('c41_idx', col('c41_idx')\
                                   .cast('float')).orderBy('prediction')

# Select only prediction and label columns
preds_and_labels  = preds_and_labels.select(['prediction', 'c41_idx'])

In [27]:
acc_evaluator.evaluate(preds_and_labels)

0.9915381741439722

In [28]:
from pyspark.mllib.evaluation import MulticlassMetrics

In [29]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [30]:
# Confusion matrix
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

In [31]:
print(metrics.confusionMatrix().toArray())

[[5.6551e+04 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 2.1377e+04 3.1000e+01 0.0000e+00 1.0000e+00 0.0000e+00
  1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [1.0000e+00 0.0000e+00 1.9259e+04 1.0000e+00 4.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 1.4000e+01 4.7100e+02 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+

In [32]:
import pandas as pd
pd.set_option('display.max_columns', 30)

In [33]:
matrix = pd.DataFrame(metrics.confusionMatrix().toArray())

In [34]:
matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,56551.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,21377.0,31.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,19259.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,14.0,471.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,32.0,0.0,280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,259.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,197.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,5.0,0.0,0.0,169.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,222.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
# With accuracy, this model is very good.
# But with confusion matrix: Some classes, which only have a few samples, have wrong predictions

## Make new prediction

In [36]:
# New data
data_new = spark.read.csv('Cung cap du lieu buoi 6/kdd_data/kddcup.testdata.unlabeled_10_percent.gz', 
                          inferSchema=True,
                          header=False)

In [37]:
data_new.count()

311029

In [38]:
str(data_new.columns)

"['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22', '_c23', '_c24', '_c25', '_c26', '_c27', '_c28', '_c29', '_c30', '_c31', '_c32', '_c33', '_c34', '_c35', '_c36', '_c37', '_c38', '_c39', '_c40']"

In [39]:
# Make predictions on the testing data
predictions_new = pipeline.transform(data_new)

In [40]:
predictions_new.select('features', 'prediction').show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|(115,[4,68,78,79,...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[8,68,78,96,...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[2,5,68,78,7...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[2,5,68,78,7...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[0,2,6,68,78...|       2.0|
|(115,[2,5,68,78,7...|       2.0|
|(115,[2,5,68,78,7...|       2.0|
|(115,[2,5,68,78,7...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[4,68,78,79,...|       2.0|
+--------------------+----------+
only showing top 20 rows

